In [1]:
import pandas as pd

In [ ]:
'''If executed in Google Colab, uncomment the following lines'''
#from google.colab import drive
#drive.mount('/content/drive')

#import os
#os.chdir('/content/drive/MyDrive/LLM_CreditorRRPrediction')

In [2]:
transcript = pd.read_csv('transcripts/transcripts.csv', delimiter='|')
qna =  pd.read_csv('transcripts/QnA.csv', delimiter='|')

# Merge the two dataframes
df = pd.merge(transcript, qna[['transcript','filename']], on='filename')

# rename transcript_x to presentation and transcript_y to QnA
df.rename(columns = {'transcript_x':'presentation', 'transcript_y':'QnA'}, inplace = True)

In [3]:
mapping = pd.read_csv('data/mapping.csv')

# create a new column 'AllNames' that concatenates all versions of 'Company' for a 'CompanyName'
mapping['AllNames'] = mapping.groupby('RR_CompanyName')['Transcript_Mapping'].transform(lambda x: ', '.join(x))
mapping.head()

,RR_CompanyName,Transcript_Mapping,AllNames
0,"1-800-FLOWERS.COM, Inc.",1-800-Flowers.com Inc.,1-800-Flowers.com Inc.
1,3M Company,3M Company,"3M Company, 3M Co."
2,3M Company,3M Co.,"3M Company, 3M Co."
3,A.M. Castle & Co.,A. M. Castle Co.,"A. M. Castle Co., A.M. Castle Co., AM Castle..."
4,A.M. Castle & Co.,A.M. Castle Co.,"A. M. Castle Co., A.M. Castle Co., AM Castle..."


In [4]:
# Load recovery rates
rr = pd.read_csv('data/RR_Bonds.csv')
rr = rr[['Ddate', 'RR', 'CompanyName', 'CUSIP', 'LTDIssuance2', 'Intangibility', 'Receivables1']]

preprocessed_df = pd.read_csv('data/preprocessed_bond_data.csv')

# Add rr columns to preprocessed_df on index
preprocessed_df['RR'] = rr['RR']
preprocessed_df['Ddate'] = rr['Ddate']
preprocessed_df['CompanyName'] = rr['CompanyName']
preprocessed_df['CUSIP'] = rr['CUSIP']
preprocessed_df['LTDIssuance2'] = rr['LTDIssuance2']
preprocessed_df['Intangibility'] = rr['Intangibility']
preprocessed_df['Receivables1'] = rr['Receivables1']

rr = preprocessed_df

# Convert 'Date' column to datetime
rr['Ddate'] = pd.to_datetime(rr['Ddate'], errors='coerce')
rr.head()

,RR,ActIndustryDistress1,ActIndustryDistress2,Senior secured,Senior unsecured,Senior subordinated,Subordinated \& Junior,Equity value,Default barrier,Net income margin,...,Russell 2000 Price Index return,Russell 2000 Vol 1m,Wilshire US Small-Cap Price Index,Wilshire Small Cap Vol,Ddate,CompanyName,CUSIP,LTDIssuance2,Intangibility,Receivables1
0,0.189010,0,0,0,1,0,0,1.28712,0.258205,-0.776257,...,0.01903,21.04,3056.03,808.357714,2004-01-01,Bethlehem Steel Corp.,087509AL9,0.467834468,0.058009127,0.029416454
1,20.553472,0,1,0,1,0,0,-135.21500,1.269706,-0.564199,...,0.01903,21.04,3137.10,974.749210,2004-05-01,"T-Mobile US, Inc.",45071TAD7,0,0.200428895,0.032214499
2,54.315958,0,1,0,1,0,0,-366.57500,1.081883,-0.671751,...,0.01903,21.05,3178.04,825.987663,2004-01-15,RCN Corporation,749361AC5,0,0.005146611,0.032214499
3,54.798870,0,1,0,1,0,0,-366.57500,1.081883,-0.671751,...,0.01903,21.05,3178.04,825.987663,2004-01-15,RCN Corporation,749361AD3,0,0.005146611,0.029416454
4,56.666288,0,1,0,1,0,0,-366.57500,1.081883,-0.671751,...,0.01903,21.05,3178.04,825.987663,2004-01-15,RCN Corporation,749361AG6,0,0.005146611,0.029416454


In [5]:
# merge rr with mapping on CompanyName and RR_CompanyName
rr = rr.merge(mapping, left_on='CompanyName', right_on='RR_CompanyName')

In [ ]:
'''Get last earnings call before default'''

# join with df on Company and Transcripts_Mapping
merged_df = rr.merge(df, left_on='Transcript_Mapping', right_on='Company')
print(merged_df['CompanyName'].value_counts())

# Ensure the columns are in datetime format
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
merged_df['Ddate'] = pd.to_datetime(merged_df['Ddate'])

merged_df['t_delta'] = merged_df['Date'] - merged_df['Ddate']

# Filter out rows where the Date is greater than the Ddate
merged_df = merged_df[merged_df['Ddate']>=merged_df['Date']]
# Get the last row for each CUSIP
merged_df = merged_df.sort_values(by='Date').groupby(['CUSIP']).tail(1)

print(merged_df['CompanyName'].value_counts())

merged_df.reset_index(drop=True, inplace=True)
# Create an ID based on unique CompanyName and Date
merged_df['call_ID'] = merged_df.groupby(['Date','CompanyName']).ngroup()

print(merged_df['call_ID'].nunique())

Ally Financial Inc.               10317
CIT Group Inc.                    10185
Lehman Brothers Holdings, Inc.     2853
Charter Communications, Inc.       2144
Sempra Energy                      1147
                                  ...  
Frontier Group Holdings, Inc.         1
Dayton Superior Corporation           1
Franklin Bank Corp.                   1
Kellwood Company, LLC                 1
Turning Point Brands, Inc.            1
Name: CompanyName, Length: 210, dtype: int64
Lehman Brothers Holdings, Inc.      317
CIT Group Inc.                      291
Charter Communications, Inc.         28
Ford Motor Company                   19
iStar Inc.                           17
                                   ... 
Centrus Energy Corp.                  1
Education Management Corporation      1
Exelon Corporation                    1
Venoco, Inc.                          1
Kellwood Company, LLC                 1
Name: CompanyName, Length: 159, dtype: int64


In [11]:
# export the merged_df to a csv file
merged_df.to_csv('transcripts/credit_df.csv', index=False, sep='|')

In [ ]:
aggregated_df = merged_df[['call_ID', 'presentation', 'QnA']].drop_duplicates().sort_values('call_ID')
aggregated_df.reset_index(drop=True, inplace=True)

# export the aggregated_df to a csv file
aggregated_df.to_csv('transcripts/aggregated_credit_df.csv', index=False, sep='|')

In [14]:
'''Get first earnings call after default'''
'''Within first 30 days'''

# join with df on Company and Transcripts_Mapping
merged_df = rr.merge(df, left_on='Transcript_Mapping', right_on='Company')
print(merged_df['CompanyName'].value_counts())

# Ensure the columns are in datetime format
merged_df['Date'] = pd.to_datetime(merged_df['Date'])
merged_df['Ddate'] = pd.to_datetime(merged_df['Ddate'])

merged_df['t_delta'] = merged_df['Date'] - merged_df['Ddate']

# Filter out rows where the Date is greater than the Ddate
merged_df = merged_df[merged_df['Ddate']<merged_df['Date']]
merged_df = merged_df[merged_df['t_delta'] <= pd.Timedelta(days=30)]
# Get the last row for each CUSIP
merged_df = merged_df.sort_values(by='Date').groupby(['CUSIP']).head(1)

print(merged_df['CompanyName'].value_counts())

merged_df.reset_index(drop=True, inplace=True)
# Create an ID based on unique CompanyName and Date
merged_df['call_ID'] = merged_df.groupby(['Date','CompanyName']).ngroup()

print(merged_df['call_ID'].nunique())

Ally Financial Inc.               10317
CIT Group Inc.                    10185
Lehman Brothers Holdings, Inc.     2853
Charter Communications, Inc.       2144
Sempra Energy                      1147
                                  ...  
Frontier Group Holdings, Inc.         1
Dayton Superior Corporation           1
Franklin Bank Corp.                   1
Kellwood Company, LLC                 1
Turning Point Brands, Inc.            1
Name: CompanyName, Length: 210, dtype: int64
Lehman Brothers Holdings, Inc.                47
iStar Inc.                                    15
Sempra Energy                                 13
JPMorgan Chase & Co.                          11
Hovnanian Enterprises, Inc.                    6
Nexstar Media Group, Inc.                      5
WestRock Company                               4
Textron Inc.                                   3
Battalion Oil Corporation                      3
Greif, Inc.                                    3
MGM Resorts International

In [15]:
# export the merged_df to a csv file
merged_df.to_csv('transcripts/post_credit_df.csv', index=False, sep='|')

In [17]:
aggregated_df = merged_df[['call_ID', 'presentation', 'QnA']].drop_duplicates().sort_values('call_ID')
aggregated_df.reset_index(drop=True, inplace=True)

# export the aggregated_df to a csv file
aggregated_df.to_csv('transcripts/post_aggregated_credit_df.csv', index=False, sep='|')